In [1]:
from dataclasses import dataclass, field
from vi import Agent, Config, Simulation, Vector2
import random
random.seed(777)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
@dataclass
class FlockingConfig(Config):
    alignment_weight: float = 1.1
    cohesion_weight: float = 0.03
    separation_weight: float = 3.1
    wind_strength: float = 0.0
    wind_direction: Vector2 = field(default_factory=lambda: Vector2(1, 0))

class FlockingAgent(Agent):
    def change_position(self):
        self.there_is_no_escape()

        neighbors = list(self.in_proximity_accuracy())

        if not neighbors:
            self.pos += self.move 
            return
        
        separationVelocity = Vector2() # move away from nearby agents
        alignmentVelocity = Vector2() # match the average direction of nearby agents
        cohesionVelocity = Vector2() # move toward the average position of nearby agents
        num_neighbors = len(neighbors)

        for neighbor, distance in neighbors: # loop trough each nearby agent.
            offset = self.pos - neighbor.pos # a vector pointing away from the neighbor
            if distance > 0:
                separationVelocity += offset / (distance**1.3) # push neighbors away

            alignmentVelocity += neighbor.move # sum al neighbors movement direction
            cohesionVelocity += neighbor.pos # sum all neigbors position

        # take the average of each vector to get a single direction fro each behavior
        separationVelocity /= num_neighbors
        alignmentVelocity /= num_neighbors
        cohesionVelocity = (cohesionVelocity / num_neighbors - self.pos)

        # multiply bt weights
        separationVelocity *= self.config.separation_weight
        alignmentVelocity *= self.config.alignment_weight
        cohesionVelocity *= self.config.cohesion_weight

        windStrengthMutation = self.config.wind_strength + random.randint(-1, 1) * 0.0001
        self.config.wind_strength = max(0, windStrengthMutation)
        windDirectionMutation = self.config.wind_direction.rotate(random.randint(-5, 5))
        self.config.wind_direction = windDirectionMutation.normalize()
        windVelocity = windDirectionMutation * windStrengthMutation if windStrengthMutation > 0 else Vector2(0, 0)
        
        # single movement vector for all behaviors
        self.move = (separationVelocity + alignmentVelocity + cohesionVelocity).normalize() * self.config.movement_speed
        self.pos += windVelocity
        self.pos += self.move

(
    Simulation(
        FlockingConfig(image_rotation = True, movement_speed = 2.0, radius = 50, seed = 777, duration = 10000, fps_limit = 60)
    )
    .batch_spawn_agents(100, FlockingAgent, images=["images/triangle.png"])
    .run()
)

: 